In [ ]:
import collections
import os
import random
import sys
import traceback
import uuid
from datetime import datetime, timedelta
from flask import Flask, redirect, url_for, render_template

import firebase_admin
from firebase_admin import firestore
from flask import Flask, render_template, request, jsonify

from flask_session import Session

In [791]:
LANGUAGE = 'hebrew'

In [792]:
# Initialize Firestore DB
cred = firebase_admin.credentials.Certificate('../key.json')
# default_app = firebase_admin.initialize_app(cred)
# 
cred = firebase_admin.credentials.Certificate('../key.json')
db = firestore.client()
summaries_ref = db.collection(LANGUAGE)
info_ref = db.collection('info')


summaries_ref_data = list(map(lambda t: t.to_dict(), list(summaries_ref.get())))
# Create a collection for the language
info_ref_data = list(map(lambda t: t.to_dict(), list(info_ref.get())))

logs = db.collection('logs')
global_variables_db = db.collection('global_variables')

# Statistics 

In [793]:
# Get the status of specific language 

from collections import Counter
language_doc = info_ref.document(LANGUAGE).get().to_dict()
value_count = Counter(language_doc.values())
value_count

Counter({2: 28, 3: 35, 1: 23, 0: 14})

In [795]:
workers = list(set([summary['work_id'] for summary in summaries_ref_data]))
print(f"Unique Workers: {workers}, length {len(workers)}")

# for summary in summaries_ref_data:
    

Unique Workers: ['1C', '1A', '1B', '1D'], length 4


In [727]:
from datetime import datetime
start_time = datetime.fromisoformat(summaries_ref_data[0]['date_start'])
end_time = datetime.fromisoformat(summaries_ref_data[0]['date_finish'])
elapsed = start_time - end_time

In [733]:
summaries_with_enough_time = []
for summary in summaries_ref_data:
    start_time = datetime.fromisoformat(summary['date_start'])
    end_time = datetime.fromisoformat(summary['date_finish'])
    if end_time - start_time > timedelta(seconds=60):
        summaries_with_enough_time.append(summary)

In [750]:
import collections
worker_to_summaries = collections.defaultdict(list)
for summary in summaries_with_enough_time:
    worker_to_summaries[summary['work_id']].append(summary)

In [767]:
len(worker_to_summaries['A3EBN8D8YBZ205'])

42

In [768]:
final_summaries = []
for summary in summaries_with_enough_time:
    if summary['work_id'] in ['A3VTGT5NZT9A4K','A3EBN8D8YBZ205',
 'A2DJ2UT4M4X5BO',
 'A1OWM3QO7QAKBN']:
        final_summaries.append(summary)

### Clean Data

In [745]:
# Get all the workers for specific language
from google.cloud.firestore_v1.base_query import FieldFilter

workers = list(set([summary['work_id'] for summary in summaries_with_enough_time]))
workers

['A2ZPPP0XVV8C0W',
 'A3EBN8D8YBZ205',
 'A1KC11A24E1WZ5',
 'A3VTGT5NZT9A4K',
 'A2DJ2UT4M4X5BO',
 'A1OWM3QO7QAKBN']

In [ ]:
# Query documents where 'spanish' is a key
from google.cloud.firestore_v1.base_query import FieldFilter

query = info_ref.where(filter=FieldFilter("ID", "==", "spanish"))
docs = query.get()


In [ ]:
docs = info_ref.get()

# Create a set to store all unique fields across documents
all_fields = set()

# Iterate over each document and collect field names
for doc in docs:
    doc_fields = doc.to_dict().keys()  # Get all keys (field names) in the document
    all_fields.update(doc_fields)      # Add these keys to the set of all fields

In [ ]:
info_ref

In [797]:
import pandas as pd
from collections import defaultdict

# Example list of dictionaries (your data)
data = final_summaries

# Specify the keys you want to include in the DataFrame
keys_to_include = ['coherence_gpt', 'coherence_gpt', 'fluency_gpt', 'relevance_gpt', 'coherence_gemini', 'coherence_gemini', 'fluency_gemini', 'relevance_gemini', 'gpt_prediction', 'gemini_prediction', 'gemini_prediction', 'summary']

# Step 1: Group the dictionaries by 'n_sample'
grouped_data = defaultdict(lambda: defaultdict(list))

for entry in data:
    n_sample = entry['n_sample']
    for key, value in entry.items():
        if key in keys_to_include:
            grouped_data[n_sample][key].append(value)

# Step 2: Convert the grouped data to a list of dicts suitable for DataFrame creation
df_data = []

for n_sample, values_dict in grouped_data.items():
    # Prepare a dictionary for this row
    row = {'q_idx': n_sample}
    for key, values in values_dict.items():
        if key in keys_to_include:
            row[key] = values
    df_data.append(row)

# Step 3: Convert the list of dicts into a DataFrame
df = pd.DataFrame(df_data)

df.to_csv(f"/Users/itaimondshine/PycharmProjects/NLP/eval_metrics/app/statistics/human/{LANGUAGE}.csv")

# Show the DataFrame
print(df)


    q_idx                                     gpt_prediction relevance_gpt  \
0      99  [نيللي، المطرب الشهير، تم اعتقاله بتهمة الاغتص...     [5, 3, 4]   
1      27  [في اليونان، اصطف الناس أمام المصارف خوفًا من ...           [4]   
2       7  [تم قتل أريغوني، ناشط إيطالي، على يد جماعة سلف...        [2, 5]   
3      26  [تصريحات الأمين العام بان كي مون جاءت بعد محاد...     [5, 3, 3]   
4      55  [نفى المتحدث باسم الرئيس الروسي مزاعم تجسس روس...           [4]   
..    ...                                                ...           ...   
80     68  [تعاقب الإنجليز والاسكتلنديون على السيطرة على ...           [5]   
81      6  [يتحدث النص عن بناء جدار في قرية بتير الفلسطين...           [5]   
82     83  [أكد وزير الخارجية السعودي، عادل الجبير، التزا...        [2, 2]   
83     92  [تتحدث النص عن تطورات لقاح كوفيد-19 في الصين، ...        [3, 3]   
84     79  [رئيس الوزراء الإثيوبي، آبي أحمد، يعلن أن أي ز...           [4]   

                                              summary coherence

## Update the Dict 

In [803]:
from collections import defaultdict

# Example list of dictionaries (your data)
data = unique_data

# Step 1: Create a dictionary to store the count of each n_sample
n_sample_counts = defaultdict(int)

# Step 2: Count the occurrences of each n_sample
for entry in data:
    n_sample = str(entry['n_sample'])
    n_sample_counts[n_sample] += 1

# Convert to a regular dictionary if you don't want the defaultdict behavior
n_sample_counts = dict(n_sample_counts)

# Show the resulting dictionary
print(n_sample_counts)

{'17': 2, '96': 1, '84': 2, '7': 1, '4': 2, '87': 2, '64': 2, '73': 2, '67': 3, '29': 2, '33': 3, '58': 3, '94': 2, '72': 3, '42': 3, '21': 3, '34': 2, '88': 1, '20': 1, '59': 2, '35': 2, '1': 1, '48': 1, '76': 1, '79': 1, '18': 1, '66': 2, '28': 1, '85': 1, '62': 1, '32': 1, '49': 2, '27': 1, '92': 1, '26': 1, '75': 1, '40': 1, '50': 2, '95': 2, '89': 2, '10': 2, '31': 1, '60': 2, '47': 1, '55': 2, '11': 2, '41': 2, '8': 1, '51': 1, '68': 3, '6': 2, '81': 2, '63': 2, '3': 2, '70': 1, '46': 1, '86': 2, '93': 2, '24': 2, '15': 2, '12': 1, '90': 2, '56': 2, '97': 1, '98': 1, '61': 1, '5': 2, '74': 1, '44': 2, '65': 1, '16': 2, '14': 1, '36': 1, '30': 1, '13': 1, '77': 1, '23': 1, '71': 1, '52': 1, '38': 1, '25': 1, '54': 1, '57': 1, '37': 1, '43': 1}


In [800]:
seen = set()

# Filter the list to remove duplicates
unique_data = []
for item in summaries_ref_data:
    # Create a tuple of work_id and n_sample
    identifier = (item['work_id'], item['n_sample'])
    if identifier not in seen:
        seen.add(identifier)
        unique_data.append(item)

print(unique_data)

[{'gpt_prediction': 'איל ההון, בעל העיתון "ישראל היום", נתן ראיון בו הכחיש את הטענות כי העיתון הוקם ומופעל עבור ראש הממשלה בנימין נתנהיהו. הוא טען כי העיתון הוקם כדי לתת לישראל חדשות ודעות מנקודת מבט הוגנת ומאוזנת. הוא גם הביע ביקורת על העיתונים המתחרים, "מעריב" ו"ידיעות אחרונות", וטען כי הם מנסים להשפיע על דעת הקהל באמצעות דיווחים מטעויות ומפלצות עליו ועל העיתון שלו. הוא גם הביע ביקורת על עורך העיתון "ידיעות אחרונות", ארנון מוזס, וטען כי הוא מנסה לעצב את דעת הקהל ולקדם את האג\'נדה הכלכלית הפרטית שלו.', 'answer': 'אין קשר', 'relevance_gpt': '1', 'gpt_question': 'מה הקשר בין הדחייה של יוזמתו לחיבה הבולטת כלפיו?', 'summary': 'על רקע הצעת חוק שנועדה למנוע ממנו להחזיק ב"ישראל היום", טוען איל ההון היהודי-אמריקאי שלדון אדלסון כי לא הקים את העיתון כדי לעזור לבנימין נתניהו ותוקף את מו"ל "ידיעות אחרונות" ארנון (נוני) מוזס ', 'gpt_grade': 4, 'consistence_gpt': '1', 'work_id': '1A', 'assignmentId': '14', 'task': 0, 'key': '1A140', 'hit_id': '14', 'date_start': '2024-08-30 14:18:17.228612', 'coher

In [553]:
additional_values = {str(i):0 for i in range(100,150)}

In [554]:
n_sample_counts.update(additional_values)

In [555]:
summary_ref = info_ref.document(LANGUAGE)

In [805]:
summary_ref.set(n_sample_counts)

update_time {
  seconds: 1726140238
  nanos: 192339000
}

In [ ]:
[_ for _ in summaries_ref_data if _['work_id'] == 'AVW7ZRR5PLRFW' and _['task'] == 9]

In [ ]:
# Calculate Agreement Rate

In [459]:
# Initialize Firestore DB
LANGUAGE = 'chinese'
cred = firebase_admin.credentials.Certificate('../key.json')
# default_app = firebase_admin.initialize_app(cred)
# 
cred = firebase_admin.credentials.Certificate('../key.json')
db = firestore.client()
summaries_ref = db.collection(LANGUAGE)
info_ref = db.collection('info')


summaries_ref_data = list(map(lambda t: t.to_dict(), list(summaries_ref.get())))
# Create a collection for the language
info_ref_data = list(map(lambda t: t.to_dict(), list(info_ref.get())))



logs = db.collection('logs')
global_variables_db = db.collection('global_variables')

In [783]:
%%capture

import pandas as pd
import numpy as np

# Assuming your data is in a list called 'data'
data = final_summaries

# Find unique n_sample values
unique_n_samples = list(set(item['n_sample'] for item in data))

# Create DataFrame with all columns
df = pd.DataFrame(columns=['fluency_gemini', 'fluency_gpt', 'coherence_gemini', 'coherence_gpt',
                           'relevance_gemini', 'relevance_gpt', 'consistence_gemini', 'consistence_gpt'])

for n_sample in unique_n_samples:
    filtered_data = [item for item in data if item['n_sample'] == n_sample]
    print(filtered_data)
    print(filtered_data)
    row_data = {
        'fluency_gemini':[int(item['fluency_gemini']) for item in filtered_data],
        'fluency_gpt': [int(item['fluency_gpt']) for item in filtered_data],
        'coherence_gemini': [int(item['coherence_gemini']) for item in filtered_data],
        'coherence_gpt': [int(item['coherence_gpt']) for item in filtered_data],
        'relevance_gemini': [int(item['relevance_gemini']) for item in filtered_data],
        'relevance_gpt': [int(item['relevance_gpt']) for item in filtered_data],
        'consistence_gemini': [int(item['consistence_gemini']) for item in filtered_data],
        'consistence_gpt': [int(item['consistence_gpt']) for item in filtered_data],
        # 'n_sample': n_sample
    }
    
    df = df.append(row_data, ignore_index=True)

# Add a new column 'n_sample' with the corresponding values


# Create a new DataFrame with average values
# df = df.dropna()
# df_averages = df.applymap(lambda x: sum(x) / len(x))
# df_averages['q_idx'] = list(unique_n_samples)
# 
# print(df_averages)

In [784]:
df

,fluency_gemini,fluency_gpt,coherence_gemini,coherence_gpt,relevance_gemini,relevance_gpt,consistence_gemini,consistence_gpt
0,[5],[5],[5],[5],[5],[5],[5],[5]
1,"[4, 4, 5]","[4, 4, 4]","[4, 3, 5]","[4, 3, 5]","[4, 4, 4]","[4, 4, 4]","[4, 4, 4]","[4, 4, 4]"
2,"[3, 4]","[4, 4]","[2, 4]","[4, 3]","[3, 4]","[3, 2]","[2, 3]","[3, 2]"
3,"[4, 4]","[4, 4]","[4, 4]","[4, 4]","[5, 3]","[5, 3]","[5, 3]","[5, 3]"
4,[5],[5],[5],[5],[4],[5],[5],[5]
...,...,...,...,...,...,...,...,...
80,[4],[4],[3],[3],[3],[3],[3],[3]
81,"[3, 4]","[3, 4]","[3, 4]","[3, 4]","[4, 4]","[4, 4]","[4, 4]","[4, 4]"
82,"[4, 3, 3]","[4, 3, 3]","[4, 4, 3]","[4, 4, 3]","[5, 4, 2]","[5, 4, 2]","[5, 4, 3]","[5, 3, 3]"
83,"[2, 3, 3]","[2, 3, 3]","[2, 2, 4]","[2, 2, 4]","[2, 2, 4]","[2, 2, 4]","[2, 2, 3]","[2, 2, 4]"


In [699]:
df = pd.DataFrame(df['fluency_gemini'])

In [785]:
df_single_column = df.melt(value_name='stacked').sort_index()

# Drop the variable column (which was holding the original column names)
df_single_column = df_single_column[['stacked']].reset_index(drop=True)

print(df_single_column)

       stacked
0          [5]
1    [4, 4, 5]
2       [3, 4]
3       [4, 4]
4          [5]
..         ...
675        [3]
676     [4, 4]
677  [5, 3, 3]
678  [2, 2, 4]
679  [5, 3, 4]

[680 rows x 1 columns]


In [786]:
df_single_column = df_single_column[df_single_column['stacked'].apply(len) == 3]

In [787]:
df_single_column

,stacked
1,"[4, 4, 5]"
7,"[4, 4, 5]"
8,"[4, 4, 2]"
10,"[4, 4, 2]"
14,"[4, 4, 3]"
...,...
670,"[4, 3, 3]"
671,"[5, 2, 4]"
677,"[5, 3, 3]"
678,"[2, 2, 4]"


In [775]:

# Function to pad lists to length 3 with None
def pad_list(x, length=3):
    return x + [None] * (length - len(x))

# Apply the padding function to the DataFrame column
df_single_column['stacked'] = df_single_column['stacked'].apply(pad_list)



In [781]:
df_single_column

,stacked
0,"[5, None, None]"
1,"[4, 4, 5]"
2,"[3, 4, None]"
3,"[4, 4, None]"
4,"[5, None, None]"
...,...
675,"[3, None, None]"
676,"[4, 4, None]"
677,"[5, 3, 3]"
678,"[2, 2, 4]"


In [788]:
import numpy as np
import pandas as pd

# Sample DataFrame
# data = {
#     'stacked': [
#         [5, 4],
#         [4, 4],
#         [3, 4],
#         [4, 3],
#         [4, None],  # After padding
#         [3, 3],
#         [4, 4],
#         [5, 3],
#         [2, 2],
#         [5, 3]
#     ]
# }

df =df_single_column

# Convert the column of lists into a 3xN matrix
matrix = np.array(df['stacked'].tolist()).T

# Function to replace None with *
def replace_none_with_star(x):
    return " ".join(str(i) if i is not None else "*" for i in x)

# Convert each row of the matrix to the desired format
formatted_data = tuple(replace_none_with_star(row) for row in matrix)

# Print the formatted data in the desired format
print(f"data = {formatted_data}")


data = ('4 4 4 4 4 4 4 4 1 3 3 3 3 4 4 5 4 4 4 3 3 4 3 4 4 4 2 4 4 4 4 3 4 4 2 4 4 4 4 4 4 4 4 4 1 3 3 4 3 4 4 2 4 4 4 3 3 4 3 4 4 4 2 4 4 4 4 3 4 4 2 4 4 4 3 2 4 3 3 5 1 3 2 3 3 4 4 5 4 4 3 2 4 4 3 4 3 2 4 3 4 3 3 3 4 4 2 4 4 4 3 2 4 3 3 4 1 3 2 4 3 4 4 4 4 4 3 2 4 4 3 4 4 4 4 3 4 3 3 3 4 4 2 4 4 4 3 2 4 3 3 5 2 3 4 4 3 4 5 5 3 3 4 3 4 3 3 4 4 2 4 3 4 4 4 4 4 5 2 5 4 4 3 2 4 3 3 5 2 3 3 4 3 4 5 4 3 3 4 3 4 3 3 4 4 4 4 3 4 4 4 4 4 5 2 5 4 4 4 3 5 3 3 5 2 2 4 4 2 3 3 5 4 4 5 4 4 3 2 4 4 2 4 2 4 4 2 4 5 5 2 5 4 4 4 3 5 3 3 5 1 2 4 4 2 3 3 2 4 4 5 4 4 3 2 4 4 3 4 2 4 4 2 4 5 5 2 5', '4 4 4 4 4 4 4 4 3 4 3 4 4 4 3 4 3 3 3 4 3 3 4 4 3 4 3 4 4 4 3 4 3 3 3 4 4 3 4 4 4 4 4 4 3 4 3 4 4 4 3 4 3 3 4 4 3 3 4 4 3 4 3 3 4 4 4 4 2 3 3 4 3 4 4 4 4 3 3 4 4 4 3 4 4 2 2 4 2 3 3 4 3 4 4 3 2 4 3 3 4 5 3 4 2 4 2 4 3 4 4 4 4 3 3 4 4 4 3 4 4 2 2 4 2 3 4 4 3 4 4 3 2 4 3 3 4 5 4 4 2 4 2 4 4 3 3 3 3 3 3 3 4 4 3 5 4 2 2 4 2 3 4 4 4 4 4 3 2 4 2 2 4 5 4 3 2 4 2 3 4 3 3 3 3 3 3 3 4 4 3 5 4 2 2 4 2 3 4 4 4 4 4 3 2 4 

In [789]:
len(formatted_data[2])

575

In [790]:

try:
    import numpy as np
except ImportError:
    np = None


def nominal_metric(a, b):
    return a != b


def interval_metric(a, b):
    return (a-b)**2


def ratio_metric(a, b):
    return ((a-b)/(a+b))**2


def krippendorff_alpha(data, metric=interval_metric, force_vecmath=False, convert_items=float, missing_items=None):
    '''
    Calculate Krippendorff's alpha (inter-rater reliability):
    
    data is in the format
    [
        {unit1:value, unit2:value, ...},  # coder 1
        {unit1:value, unit3:value, ...},   # coder 2
        ...                            # more coders
    ]
    or 
    it is a sequence of (masked) sequences (list, numpy.array, numpy.ma.array, e.g.) with rows corresponding to coders and columns to items
    
    metric: function calculating the pairwise distance
    force_vecmath: force vector math for custom metrics (numpy required)
    convert_items: function for the type conversion of items (default: float)
    missing_items: indicator for missing items (default: None)
    '''
    
    # number of coders
    m = len(data)
    
    # set of constants identifying missing values
    if missing_items is None:
        maskitems = []
    else:
        maskitems = list(missing_items)
    if np is not None:
        maskitems.append(np.ma.masked_singleton)
    
    # convert input data to a dict of items
    units = {}
    for d in data:
        try:
            # try if d behaves as a dict
            diter = d.items()
        except AttributeError:
            # sequence assumed for d
            diter = enumerate(d)
            
        for it, g in diter:
            if g not in maskitems:
                try:
                    its = units[it]
                except KeyError:
                    its = []
                    units[it] = its
                its.append(convert_items(g))


    units = dict((it, d) for it, d in units.items() if len(d) > 1)  # units with pairable values
    n = sum(len(pv) for pv in units.values())  # number of pairable values
    
    if n == 0:
        raise ValueError("No items to compare.")
    
    np_metric = (np is not None) and ((metric in (interval_metric, nominal_metric, ratio_metric)) or force_vecmath)
    
    Do = 0.
    for grades in units.values():
        if np_metric:
            gr = np.asarray(grades)
            Du = sum(np.sum(metric(gr, gri)) for gri in gr)
        else:
            Du = sum(metric(gi, gj) for gi in grades for gj in grades)
        Do += Du/float(len(grades)-1)
    Do /= float(n)

    if Do == 0:
        return 1.

    De = 0.
    for g1 in units.values():
        if np_metric:
            d1 = np.asarray(g1)
            for g2 in units.values():
                De += sum(np.sum(metric(d1, gj)) for gj in g2)
        else:
            for g2 in units.values():
                De += sum(metric(gi, gj) for gi in g1 for gj in g2)
    De /= float(n*(n-1))

    return 1.-Do/De if (Do and De) else 1.

missing = '*' # indicator for missing values
array = [d.split() for d in formatted_data]  # convert to 2D list of string items

print("nominal metric: %.3f" % krippendorff_alpha(array, nominal_metric, missing_items=missing))
print("interval metric: %.3f" % krippendorff_alpha(array, interval_metric, missing_items=missing))

nominal metric: 0.010
interval metric: 0.023


In [716]:
formatted_data

('4 5 4 4 5 4 5 4 4 4 4 4 5 4 2 4 3 4 3 4 5 3 5 4 4 2 4 5 4 3 4 5 5 4 5 4 4 5 5 5 4 4 4 3 4 5 3 4 4 4 4 2 4 4 5 2 4 3 3 5 4 4 4 5 4 3 4 5 4 4 3 5 4 3 5 4 5 4 5 4 5 4 4 4',
 '5 5 5 5 4 3 4 5 4 2 5 5 4 4 5 4 4 4 1 4 4 5 5 4 4 5 4 4 4 5 4 4 5 3 4 3 5 4 4 4 5 4 5 5 5 4 5 5 5 4 4 4 5 5 5 4 4 4 4 4 5 5 4 4 5 4 5 5 4 5 2 5 4 4 4 4 4 5 5 4 4 4 4 4',
 '5 5 5 5 4 5 5 5 5 5 4 5 4 5 5 3 3 5 4 5 4 5 5 4 5 5 5 4 4 5 5 4 5 5 4 5 3 4 4 4 5 5 5 4 5 5 4 5 4 5 5 5 5 5 4 4 4 5 3 5 5 3 3 3 5 5 4 5 4 5 4 5 4 4 3 5 3 5 5 5 4 5 4 4')

In [708]:
import numpy as np
desired_length = 3

# Function to pad lists to the desired length
def pad_list(lst, length, pad_value=None):
    return lst + [pad_value] * (length - len(lst))

# Convert the DataFrame to a list of lists with padding
list_of_lists = [pad_list(lst, desired_length) for lst in df]

# Display the result
x = list_of_lists

TypeError: can only concatenate str (not "list") to str

In [506]:
x = [[2,2,2] for i in range(20)]

## Calculate Agreement Rate 

In [507]:
from itertools import chain
from numpy import transpose, identity

# This is 12x4 matrix where each row represents all 4 reviewers' ratings for one store,
# and each column represents each reviewer's ratings for all 12 store locations.
ratings = x

# Helper function to calculate the agreement table:
#   - each row i corresponds to a store
#   - each column k represents a possible rating (1, 2, 3, or 4, in this case)
#   - each table entry r_ik is the number of reviewers who gave store i rating k
def get_agreement_table(ratings, categories):
    agreement = []

    for store in ratings:
        category_counts = list(map(lambda category: store.count(category), categories))
        if sum(category_counts) > 1: # Ignore any stores with less than two ratings
            agreement.append(category_counts)

    return agreement

# Helper function that returns the weighted count of reviewers who gave store i a rating that at least partially matched category k
# (For categorical data this is just the number of reviewers who picked category k)
def get_weighted_rater_count(weights_k, agreement_i):
    weighted_count = 0
    for i in range(len(agreement_i)):
        weighted_count += weights_k[i] * agreement_i[i]
    return weighted_count

# STEPS 1 & 2: CLEANING THE DATA AND BUILDING THE AGREEMENT TABLE

# Get the set of all possible ratings our reviewer can give
rating_categories = set(chain(*ratings)) 
# Remove the placeholder value for missing ratings
# rating_categories.remove(None) 
agreement_table = get_agreement_table(ratings, rating_categories)

# n is the number of stores (more generally, the number of items being rated)
n = len(agreement_table) 
# q is the number of possible rating categories
q = len(rating_categories) 

# get an array with r_i (the total number of reviewers who rated the ith store) for all stores
raters_per_store = list(map(lambda r_ik: sum(r_ik), agreement_table)) 
# rhat is the average number of reviewers who rated each store
rhat = sum(raters_per_store) / n 

# STEP 3: CHOOSING A WEIGHT FUNCTION

# categorical weights are just the identity matrix (1 if the category matches and 0 otherwise)
weights = identity(len(rating_categories)) 

# STEP 4: CALCULATING OBSERVED PERCENT AGREEMENT (p_a)
percent_agreement = 0
for i in range(n): # Find the percent agreement for every store
    percent_agreement_i = 0
    for k in range(q): # Find the percent agreement for every category for the ith store
        rhat_ik = get_weighted_rater_count(weights[k], agreement_table[i])
        ri = sum(agreement_table[i]) # Number of people who rated this store
        percent_agreement_i_k = (agreement_table[i][k] * (rhat_ik - 1)) / (rhat * (ri - 1))
        percent_agreement_i += percent_agreement_i_k
    
    percent_agreement += percent_agreement_i

pa_prime = percent_agreement / n # Find the average store-level percent agreement

total_rating_count = n * rhat
pa = (1 - 1/(total_rating_count)) * pa_prime + 1/total_rating_count

# STEP 5: CALCULATING EXPECTED PERCENT AGREEMENT (p_e)
category_averages = list(map(lambda category: sum(category) / n, transpose(agreement_table)))
classification_probabilities = list(map(lambda category_average: category_average / rhat, category_averages))

pe = 0
for k in range(q): # For every possible pair of rating categories
    for l in range(q):
        # Add the probability of those two categories being chosen at random, weighted by how closely they match
        pe += classification_probabilities[k] * classification_probabilities[l] * weights[k][l] 

# STEP 6: CALCULATE KRIPPENDORFF'S ALPHA
alpha = (pa - pe) / (1 - pe)
print(f"Krippendorff's alpha: {alpha}")

Krippendorff's alpha: nan


/var/folders/6c/7s9j6b710cn7h_bnq4jrn_700000gn/T/ipykernel_17955/3462512869.py:81: RuntimeWarning: invalid value encountered in double_scalars
  alpha = (pa - pe) / (1 - pe)


## Correlations

In [508]:
# Load metrics df

metrics = pd.read_csv(f"/Users/itaimondshine/PycharmProjects/NLP/eval_metrics/evaluations/results/{LANGUAGE}_metrics.csv")

In [197]:
corr_with_metrics = pd.merge(metrics, df_averages, on='q_idx')
corr_with_metrics

,label,q_idx,gemini_rouge1,gemini_rouge2,gemini_rougeL,gemini_bert_f1,gpt_rouge1,gpt_rouge2,gpt_rougeL,gpt_bert_f1,fluency_gemini,fluency_gpt,coherence_gemini,coherence_gpt,relevance_gemini,relevance_gpt,consistence_gemini,consistence_gpt,n_sample
0,Ijọba apaapọ orilẹede Naijiria ni wọn yoo se i...,4,0.076433,0.000000,0.063694,0.604220,0.116667,0.025210,0.083333,0.689038,4.666667,4.000000,4.666667,4.666667,4.333333,4.333333,4.000000,4.000000,4
1,Orisirisi iwa isekupani lo n waye ni Naijiria ...,5,0.020134,0.000000,0.020134,0.525535,0.028470,0.000000,0.028470,0.627313,4.000000,3.333333,4.000000,4.000000,3.666667,4.333333,3.666667,4.000000,5
2,"Lẹyin ọ rẹyin, Mauricio Pochettino di akọnimọọ...",6,0.020339,0.000000,0.020339,0.618176,0.088889,0.000000,0.088889,0.720132,3.666667,2.333333,3.333333,3.666667,4.333333,3.333333,3.333333,3.666667,6
3,"Yoruba ni fi ọrẹ rẹ han mi, ki n sọ iru eeyan ...",9,0.056738,0.000000,0.042553,0.642652,0.095238,0.019417,0.095238,0.680629,4.000000,3.666667,3.666667,4.333333,3.666667,4.333333,4.000000,4.333333,9
4,"Ẹgbẹ oṣelu Peoples Democratic Party, PDP to n ...",14,0.028249,0.000000,0.028249,0.652458,0.270270,0.055046,0.144144,0.716088,3.333333,3.333333,3.666667,4.333333,3.666667,3.666667,4.333333,2.666667,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,"""Igbakuugba ti mo ba n wẹ, ti mo ba si ri alaf...",141,0.067669,0.007547,0.041353,0.641164,0.106583,0.000000,0.068966,0.669502,4.333333,3.333333,4.333333,4.000000,4.333333,4.000000,4.333333,3.666667,141
82,Iwa ki ọmọ maa tọ sile kii se ohun ajeji nilẹ ...,144,0.009091,0.000000,0.009091,0.633723,0.173410,0.046784,0.080925,0.712313,2.666667,3.666667,3.333333,3.666667,2.666667,2.666667,3.000000,3.000000,144
83,Ààrẹ Emmerson Mnangagwa ti jáwé olúborí níbí ì...,145,0.070175,0.000000,0.052632,0.646022,0.082305,0.008299,0.057613,0.679890,2.333333,4.333333,2.333333,5.000000,2.333333,5.000000,2.333333,5.000000,145
84,Olukọni ikọ agbabọọlu Super Eagles ti kede wi ...,146,0.057377,0.008264,0.049180,0.645040,0.214876,0.100840,0.148760,0.728950,2.666667,2.666667,2.333333,3.000000,2.666667,3.000000,3.000000,3.333333,146


In [155]:
df = corr_with_metrics

In [198]:
import pandas as pd
from scipy.stats import pearsonr

# Assuming you already have your DataFrame 'df'

# Select columns that start with 'gpt'
gpt_start_columns = [col for col in corr_with_metrics.columns if col.startswith('gpt')]

# Select columns that end with 'gpt'
gpt_end_columns = [col for col in corr_with_metrics.columns if col.endswith('gpt')]

# Initialize a DataFrame to store the results
results_df = pd.DataFrame(index=gpt_start_columns, columns=gpt_end_columns)

# Calculate the correlation and p-value between each pair of columns
for start_col in gpt_start_columns:
    for end_col in gpt_end_columns:
        corr, p_value = pearsonr(corr_with_metrics[start_col], corr_with_metrics[end_col])
        # Format the result as "Correlation (p-value)"
        results_df.loc[start_col, end_col] = f"{corr:.2f} ({p_value:.2f})"

# Display the table
print("Correlation and p-values between columns that start with 'gpt' (rows) and those that end with 'gpt' (columns):")
print(results_df)


Correlation and p-values between columns that start with 'gpt' (rows) and those that end with 'gpt' (columns):
              fluency_gpt coherence_gpt relevance_gpt consistence_gpt
gpt_rouge1    0.09 (0.42)   0.04 (0.72)   0.06 (0.59)     0.06 (0.61)
gpt_rouge2   -0.00 (0.98)  -0.06 (0.56)  -0.00 (1.00)     0.11 (0.33)
gpt_rougeL    0.07 (0.54)   0.01 (0.93)   0.04 (0.68)     0.04 (0.68)
gpt_bert_f1  -0.01 (0.93)   0.04 (0.72)   0.04 (0.70)     0.02 (0.84)


In [199]:
import pandas as pd
from scipy.stats import pearsonr

# Assuming you already have your DataFrame 'df'

# Select columns that start with 'gemini'
gemini_start_columns = [col for col in corr_with_metrics.columns if col.startswith('gemini')]

# Select columns that end with 'gemini'
gemini_end_columns = [col for col in corr_with_metrics.columns if col.endswith('gemini')]

# Initialize a DataFrame to store the results
results_df_gemini = pd.DataFrame(index=gemini_start_columns, columns=gemini_end_columns)

# Calculate the correlation and p-value between each pair of columns
for start_col in gemini_start_columns:
    for end_col in gemini_end_columns:
        corr, p_value = pearsonr(corr_with_metrics[start_col], corr_with_metrics[end_col])
        # Format the result as "Correlation (p-value)"
        results_df_gemini.loc[start_col, end_col] = f"{corr:.2f} ({p_value:.2f})"

# Save the results to a CSV file
results_df_gemini.to_csv('gemini_correlations.csv')

# Display the table
print("Correlation and p-values between columns that start with 'gemini' (rows) and those that end with 'gemini' (columns):")
print(results_df_gemini)


Correlation and p-values between columns that start with 'gemini' (rows) and those that end with 'gemini' (columns):
               fluency_gemini coherence_gemini relevance_gemini  \
gemini_rouge1     0.09 (0.43)      0.09 (0.40)      0.09 (0.43)   
gemini_rouge2     0.10 (0.37)      0.09 (0.43)      0.05 (0.66)   
gemini_rougeL     0.09 (0.41)      0.10 (0.37)      0.08 (0.46)   
gemini_bert_f1   -0.07 (0.51)     -0.03 (0.79)      0.01 (0.96)   

               consistence_gemini  
gemini_rouge1        -0.06 (0.59)  
gemini_rouge2        -0.08 (0.45)  
gemini_rougeL        -0.04 (0.69)  
gemini_bert_f1       -0.09 (0.43)  


# Workers Statistics 